# flowcatchR: A framework for tracking and analyzing flowing blood cells in time lapse microscopy images
Federico Marini, 3 February 2015 
-----------------------------

## Introduction 

*flowcatchR* is a set of tools to analyze in vivo microscopy imaging data, focused on tracking flowing blood cells. *flowcatchR* guides throughout all the steps of bioimage processing, from segmentation to calculation of features, filtering out particles not of interest, providing also a set of utilities to help checking the quality of the performed operations. The main novel contribution investigates the issue of tracking flowing cells such as the ones in blood vessels, to categorize the particles in flowing, rolling, and adherent by providing a comprehensive analysis of the identified trajectories. The extracted information is then applied in the study of phenomena such as hemostasis and thrombosis development. We expect this package to be potentially applied to a variety of assays, covering a wide range of applications founded on time-lapse microscopy.

This document offers an introduction and overview of the R/Bioconductor package *flowcatchR*, which provides a flexible and comprehensive set of tools to detect and track flowing blood cells in time-lapse microscopy. 

*flowcatchR* builds upon functionalities provided by the *EBImage* package, and extends them in order to analyze time-lapse microscopy images. Here we list some of the unique characteristics of the datasets *flowcatchR* is designed for:

- The images come from intravital microscopy experiments. This means that the Signal-to-Noise Ratio (SNR) is not optimal, and very importantly there are potential major movements of the alive specimen, that can be confounded with the true movements of the particles of interest
- Cells are densely distributed on the images, with particles that can enter and leave the field of view
- The acquisition frame rate is a compromise between allowing the fluorescent cells to be detected and detecting the movements properly  
- Cells can flow, temporarily adhere to the endothelial layer and/or be permanently adherent. Therefore, all movement modalities should be detected correctly throughout the entire acquisition. Cells can also cluster together and form (temporary) conglomerates

Essential features *flowcatchR* delivers to the user are:

- A simple and flexible, yet complete framework to analyze flowing blood cells (and more generally time-lapse) image sets , with a system of S4 classes such as `Frames`, `ParticleSet` and `TrajectorySet` constituting the backbone of the procedures
- Techniques for aiding the detection of objects in the segmentation step
- An algorithm for tracking the particles, adapted and improved from the proposal of Sbalzarini and Koumoutsakos (2005), that reflects the directional aspect of the motion
- A wide set of functions inspecting the kinematic properties of the identified trajectories, providing publication-ready summary statistics and visualization tools to help classifying identified objects

This guide includes a brief overview of the entire processing flow, from importing the raw images to the analysis of kinematic parameters derived from the identified trajectories. An example dataset will be used to illustrate the available features, in order to track blood platelets in consecutive frames derived from an intravital microscopy acquisition (also available in the package). All steps will be dissected to explore available parameters and options.

To install the package *flowcatchR*, please start a current version of R and type 


In [ ]:
source("http://bioconductor.org/biocLite.R")
biocLite("flowcatchR")

## Workflow overview

The workflow of *flowcatchR* can be summarized in a handful of commands, which are here used as template for the further analyses that could be performed.

The commands and the steps corresponding to them are explained in greater detail in the *flowcatchR* package vignette, available at http://bioconductor.org/packages/release/bioc/html/flowcatchR.html


In [ ]:
# load library and data
library("flowcatchR")
data(MesenteriumSubset)
MesenteriumSubset
fullResults <- kinematics(trajectories(particles(channel.Frames(MesenteriumSubset,"red"))))
# initialization
# fullData <- read.Frames(image.files = "/path/to/folder/containing/images/",nframes=100) 
inspect.Frames(MesenteriumSubset,nframes=4,display.method="raster")

plateletsMesenterium <- channel.Frames(MesenteriumSubset, mode="red")
inspect.Frames(plateletsMesenterium,nframes=4,display.method="raster")

# preprocessing
preprocessedPlatelets <- preprocess.Frames(plateletsMesenterium,
                                    brush.size=3, brush.shape="disc",
                                    at.offset=0.15, at.wwidth=10, at.wheight=10,
                                    kern.size=3, kern.shape="disc",
                                    ws.tolerance=1, ws.radius=1)
inspect.Frames(preprocessedPlatelets,nframes=9,display.method="raster")

# extracting the particles
platelets <- particles(plateletsMesenterium,preprocessedPlatelets)
platelets
paintedPlatelets <- add.contours(raw.frames=MesenteriumSubset,
                                 binary.frames=preprocessedPlatelets,
                                 mode="particles")
inspect.Frames(paintedPlatelets,nframes=9,display.method="raster")

# linking the particles
linkedPlatelets <- link.particles(platelets,
                                  L=26,R=3,
                                  epsilon1=0,epsilon2=0,
                                  lambda1=1,lambda2=0,
                                  penaltyFunction=penaltyFunctionGenerator(),
                                  include.area=FALSE)
linkedPlatelets
# generating the trajectories
trajPlatelets <- trajectories(linkedPlatelets)
trajPlatelets
plot(trajPlatelets,MesenteriumSubset)
plot2D.TrajectorySet(trajPlatelets,MesenteriumSubset)

paintedTrajectories <- add.contours(raw.frames=MesenteriumSubset,
                                    binary.frames=preprocessedPlatelets,
                                    trajectoryset = trajPlatelets,
                                    mode="trajectories")
inspect.Frames(paintedTrajectories,nframes=9,display.method="raster")

# computing kinematic parameters
allKinematicFeats.allPlatelets <- kinematics(trajPlatelets,
                                             trajectoryID=NULL, 
                                             acquisitionFrequency=30, 
                                             scala=50, 
                                             feature=NULL)
allKinematicFeats.allPlatelets
allVelocities <- kinematics(trajPlatelets,feature="curvilinearVelocity")
hist(allVelocities, breaks=10, probability=TRUE,col = "cadetblue",
     xlab="Curvilinear Velocities Distribution",
     main="Trajectory Analysis: Curvilinear Velocities")
lines(density(allVelocities,na.rm=TRUE),col="steelblue",lwd = 2)

# run Shiny App interactively
shinyFlow()

# snap the Frames to display interactively additional info
snap(MesenteriumSubset,preprocessedPlatelets,platelets,trajPlatelets,frameID = 1,showVelocity = T)


## Supplementary information

For more information on the method adapted for tracking cells, see Sbalzarini and Koumotsakos (2005).
For additional details regarding the functions of *flowcatchR*, please consult the documentation or write an email to (marinif@uni-mainz.de). 
For requesting help (e.g., for extending one of these analysis templates) or a particular functionality/submitting a pull request, please use https://github.com/federicomarini/flowcatchR and/or write an email to (marinif@uni-mainz.de).



## Acknowledgements
This package was developed at the Institute of Medical Biostatistics, Epidemiology and Informatics at the University Medical Center, Mainz (Germany), with the financial support provided by the TRP-A15 Translational Research Project grant. 

*flowcatchR* incorporates suggestions and feedback from the wet-lab biology units operating at the Center for Thrombosis and Hemostasis (CTH), in particular Sven Jäckel and Kerstin Jurk. Sven Jäckel also provided us with the sample acquisition which is available in the package vignette.

We would like to thank the members of the Biostatistics division for valuable discussions, and additionally Isabella Zwiener for contributing to the first ideas on the project.

## Session Information


In [ ]:
sessionInfo()
